<a href="https://colab.research.google.com/github/bpratham2001/Servilia/blob/main/notebooks/Wine_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# https://github.com/Kaggle/kagglehub/blob/main/README.md#kaggledatasetadapterpandas
#!pip install kagglehub[pandas-datasets]
!pip install opendatasets

In [41]:
import opendatasets as od
import pandas as pd
import numpy as np

#checked
#mysarahmadbhat/wine-tasting // winemag-data-130k-v2.csv \\ zynicide/wine-reviews is better

#potential sources
# https://en.wikipedia.org/wiki/Wine_tasting_descriptors
# https://www.wineenthusiast.com/toplists/wine/
# https://www.wine-tastings-guide.com/wine-descriptions.html
# https://en.wikipedia.org/wiki/Wine_tasting

od.download("https://www.kaggle.com/datasets/zynicide/wine-reviews", force=True) # winemag-data_first150k.csv, winemag-data-130k-v2.csv
file =('wine-reviews/winemag-data_first150k.csv')
df1 = pd.read_csv(file)
file =('wine-reviews/winemag-data-130k-v2.csv')
df2 = pd.read_csv(file)

od.download("https://www.kaggle.com/datasets/fedesoriano/spanish-wine-quality-dataset", force=True) # wines_SPA.csv
file =('spanish-wine-quality-dataset/wines_SPA.csv')
df3 = pd.read_csv(file)

od.download("https://www.kaggle.com/datasets/elvinrustam/wine-dataset", force=True) # WineDataset.csv
file =('wine-dataset/WineDataset.csv')
df4 = pd.read_csv(file)

Dataset URL: https://www.kaggle.com/datasets/zynicide/wine-reviews


100%|██████████| 50.9M/50.9M [00:00<00:00, 231MB/s]



Dataset URL: https://www.kaggle.com/datasets/fedesoriano/spanish-wine-quality-dataset


100%|██████████| 46.4k/46.4k [00:00<00:00, 13.2MB/s]


Dataset URL: https://www.kaggle.com/datasets/elvinrustam/wine-dataset


100%|██████████| 308k/308k [00:00<00:00, 58.0MB/s]

In [42]:
df1.rename(columns={'Unnamed: 0': 'id'}, inplace=True)
df2.rename(columns={'Unnamed: 0': 'id'}, inplace=True)
#df['DataFrame Column'] = df['DataFrame Column'].fillna(0)

In [43]:
for i in df1.keys():
  print(i +" "+ str(df1[i].isnull().values.any()))
#df1.isnull().values.any()
print("\n")
for i in df2.keys():
  print(i +" "+ str(df2[i].isnull().values.any()))

id False
country True
description False
designation True
points False
price True
province True
region_1 True
region_2 True
variety False
winery False


id False
country True
description False
designation True
points False
price True
province True
region_1 True
region_2 True
taster_name True
taster_twitter_handle True
title False
variety True
winery False


In [24]:
# fix countries in df1 and df2
df1.at[1133, 'country'] = 'Greece'
df1.at[1440, 'country'] = 'Turkey'
df1.at[68226, 'country'] = 'Chile'
df1.at[113016, 'country'] = 'Chile'
df1.at[135696, 'country'] = 'Chile'

#df2.at[?, 'country'] = '?'
countries = {'Tsililis':'Greece', 'Büyülübağ':'Turkey', 'Chilcas':'Chile',
             'Gotsa Family Wines':'Georgia', 'Barton & Guestier':'France',
             'Kakhetia Traditional Winemaking':'Georgia', 'Ross-idi':'Bulgaria',
             'Orbelus':'Bulgaria', 'St. Donat':'Hungary', 'Familia Deicas':'Uruguay','Bartho Eksteen':'South Africa',
             'Stone Castle':'Kosovo', 'Teliani Valley':'Georgia', 'Undurraga':'Chile', 'Mt. Beautiful':'New Zealand',
             'Neumeister':'Austria', 'Bachelder':'? US,CA,FR', 'Santa Ema':'Chile', 'Psagot':'Israel',
             'Newton Johnson':'South Africa', 'Ktima Voyatzi':'', 'Lismore':'', 'Logodaj':'',
             'Somlói Vándor':'', 'Les Frères Dutruy':'', 'El Capricho':'',
             'Amiran Vepkhvadze':''}

In [53]:
print(df1[df1['country'].isnull()].winery.unique())
df2[df2['country'].isnull()].winery.unique()

['Tsililis' 'Büyülübağ' 'Chilcas']


array(['Gotsa Family Wines', 'Barton & Guestier',
       'Kakhetia Traditional Winemaking', 'Tsililis', 'Ross-idi',
       'Orbelus', 'St. Donat', 'Familia Deicas', 'Bartho Eksteen',
       'Stone Castle', 'Teliani Valley', 'Undurraga', 'Mt. Beautiful',
       'Neumeister', 'Bachelder', 'Chilcas', 'Santa Ema', 'Psagot',
       'Newton Johnson', 'Ktima Voyatzi', 'Lismore', 'Logodaj',
       'Somlói Vándor', 'Les Frères Dutruy', 'El Capricho',
       'Amiran Vepkhvadze', 'Büyülübağ'], dtype=object)

In [46]:
country = set(np.concatenate((df1[df1['country'].isnull()].winery.unique(), df2[df2['country'].isnull()].winery.unique())))
len(country)

27

In [45]:
region = np.concatenate((df1[df1['region_1'].isnull()].winery.unique(), df2[df2['region_1'].isnull()].winery.unique()))
region = np.concatenate((region, df1[df1['region_2'].isnull()].winery.unique()))
region = set(np.concatenate((region, df2[df2['region_2'].isnull()].winery.unique())))
len(region)

14017

In [49]:
# reviews.loc[(reviews.country == 'Italy') & (reviews.points >= 90)]
df1.loc[(df1.region_1 == df1.region_2) & (df1.region_1.notnull())]

,id,country,description,designation,points,price,province,region_1,region_2,variety,winery
3,3,US,"This spent 20 months in 30% new French oak, an...",Reserve,96,65.0,Oregon,Willamette Valley,Willamette Valley,Pinot Noir,Ponzi
15,15,US,"First made in 2006, this succulent luscious Ch...",Sigrid,95,90.0,Oregon,Willamette Valley,Willamette Valley,Chardonnay,Bergström
27,27,US,"Focused and dense, this intense wine captures ...",Abetina,94,105.0,Oregon,Willamette Valley,Willamette Valley,Pinot Noir,Ponzi
57,57,US,"While exuberantly fruity, almost tropical on t...",Reserve,90,25.0,New York,Finger Lakes,Finger Lakes,Riesling,Dr. Konstantin Frank
73,73,US,"Massively fruity, this is an apple/pear bonanz...",NaN,91,19.0,Oregon,Willamette Valley,Willamette Valley,Pinot Gris,Trinity Vineyards
...,...,...,...,...,...,...,...,...,...,...,...
150687,150687,US,Honeyed mango and papaya flavors are accented ...,Late Harvest,86,15.0,New York,Finger Lakes,Finger Lakes,Vignoles,Wagner
150743,150743,US,"A peppery nose, tart wine, with plenty of acid...",Zin Man,83,12.0,California,Sierra Foothills,Sierra Foothills,Zinfandel,Perry Creek
150745,150745,US,"Simple fruit and a sweet, almost candied flavo...",French Camp,82,12.0,California,Central Coast,Central Coast,Zinfandel,Buffalo Ridge
150758,150758,US,"The more I taste Vidal, the more I find it an ...",Ice Wine,81,18.0,New York,Finger Lakes,Finger Lakes,Vidal Blanc,Wagner
